In [1]:
import torch
import numpy as np

## PyTorch中创建张量

### 直接创建

`torch.tensor(data, dtype=None, device=None, requires_grad=False, pin_memory=False)`

In [2]:
ndarray = np.ones((3, 3))
print("ndarray的数据类型：", ndarray.dtype)
# 创建存放在 GPU 的数据
# t = torch.tensor(ndarray, device='cuda')
t= torch.tensor(ndarray)
print(t)

ndarray的数据类型： float64
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


### 指定类型函数随机创建

In [4]:
out_a = torch.FloatTensor(2,3)
out_b = torch.IntTensor(2)
out_c = torch.IntTensor([1,2,3,4])
print(out_a,'\n',out_b,'\n',out_c)

tensor([[1.7035e-34, 0.0000e+00, 1.6143e-34],
        [0.0000e+00, 7.0062e+22, 2.1715e-18]]) 
 tensor([-1457705888,       32663], dtype=torch.int32) 
 tensor([1, 2, 3, 4], dtype=torch.int32)


### tensor和numpy array之间的相互转换

In [5]:
arr = np.array([[1, 2, 3], [4, 5, 6]])
t = torch.from_numpy(arr)
# 修改 tensor，array 也会被修改
print("\n修改tensor")
t[0, 0] = -1
print("numpy array: ", arr)
print("tensor : ", t)


修改tensor
numpy array:  [[-1  2  3]
 [ 4  5  6]]
tensor :  tensor([[-1,  2,  3],
        [ 4,  5,  6]])


注意：torch.tensor创建得到的张量和原数据是不共享内存的，张量对应的变量是独立变量。  
而torch.from_numpy()和torch.as_tensor()从numpy array创建得到的张量和原数据是共享内存的，张量对应的变量不是独立变量，修改numpy array会导致对应tensor的改变。

In [6]:
# 常见的构造Tensor的函数
k = torch.rand(2, 3) 
l = torch.ones(2, 3)
m = torch.zeros(2, 3)
n = torch.arange(0, 10, 2)
print(k, '\n', l, '\n', m, '\n', n)

tensor([[0.4110, 0.8827, 0.3000],
        [0.3471, 0.5926, 0.5523]]) 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 
 tensor([[0., 0., 0.],
        [0., 0., 0.]]) 
 tensor([0, 2, 4, 6, 8])


## PyTorch中tensor的常用方法

### 查看tensor维度信息

In [7]:
# 查看tensor的维度信息（两种方式）
out = torch.randn(2, 3)
print(out.size())
print(out.shape)

torch.Size([2, 3])
torch.Size([2, 3])


### tensor.view()

In [5]:
# 创建一个形状为 (4, 4) 的张量
x = torch.randn(4, 4)
print(x.size())  # 输出: torch.Size([4, 4])

# 使用 torch.view 将 x 重塑为形状 (16,) 的一维张量
y = x.view(16)
print(y.size())  # 输出: torch.Size([16])

# 使用 -1 自动计算维度大小，将 x 重塑为形状 (2, 8) 的二维张量
z = x.view(2, -1)
print(z.size())  # 输出: torch.Size([2, 8])

# 也可以重塑为更高维度的张量，只要元素总数保持不变
m = x.view(2, 2, 4)
print(m.size())  # 输出: torch.Size([2, 2, 4])

torch.Size([4, 4])
torch.Size([16])
torch.Size([2, 8])
torch.Size([2, 2, 4])


### 广播机制

In [6]:
x = torch.arange(1, 3).view(1, 2)
print(x)
y = torch.arange(1, 4).view(3, 1)
print(y)
# print(x + y)

tensor([[1, 2]])
tensor([[1],
        [2],
        [3]])


### CUDA中的tensor

In [7]:
x = torch.randn(1,2)
if torch.cuda.is_available():
    device = torch.device("cuda")          # cuda device对象
    y = torch.ones_like(x, device=device)  # 创建一个在cuda上的tensor
    x = x.to(device)                       # 使用方法把x转为cuda 的tensor
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # .to方法也能够同时设置类型

tensor([[1.7848, 0.6444]], device='cuda:0')
tensor([[1.7848, 0.6444]], dtype=torch.float64)


## 自动求导示例
  
>这里将通过一个简单的函数  $y=x_1+2*x_2$  来说明PyTorch自动求导的过程

In [22]:
import torch

x1 = torch.tensor(1.0, requires_grad=True)
x2 = torch.tensor(2.0, requires_grad=True)
y = x1 + 2*x2
print(y)

tensor(5., grad_fn=<AddBackward0>)


In [23]:
# 首先查看每个变量是否需要求导
print(x1.requires_grad)
print(x2.requires_grad)
print(y.requires_grad)

True
True
True


In [24]:
# 查看每个变量导数大小。此时因为还没有反向传播，因此导数都不存在
print(x1.grad.data)
print(x2.grad.data)
print(y.grad.data)

AttributeError: 'NoneType' object has no attribute 'data'

In [25]:
x1

tensor(1., requires_grad=True)

In [26]:
## 反向传播后看导数大小
y = x1 + 2*x2
y.backward()
print(x1.grad.data)
print(x2.grad.data)

tensor(1.)
tensor(2.)


In [30]:
# 导数是会累积的，重复运行相同命令，grad会增加
y = x1 + 2*x2
y.backward()
print(x1.grad.data)
print(x2.grad.data)

tensor(5.)
tensor(10.)


In [ ]:
# 所以每次计算前需要清除当前导数值避免累积，这一功能可以通过pytorch的optimizer实现。后续会讲到

In [31]:
# 尝试，如果不允许求导，会出现什么情况？
x1 = torch.tensor(1.0, requires_grad=False)
x2 = torch.tensor(2.0, requires_grad=False)
y = x1 + 2*x2
y.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn